### Load KITTI Dataset in Python

In [ ]:
import pykitti


basedir = "/home/deepak-bhagat/Downloads/data_odometry_velodyne/dataset/"


sequence = "00"  


dataset = pykitti.odometry(basedir, sequence)


print(f"Loaded {len(dataset.timestamps)} frames from sequence {sequence}")


### Visualize LiDAR Data

In [ ]:
import open3d as o3d
import numpy as np

# Load the first LiDAR frame (frame 0)
scan = dataset.get_velo(0)

# Convert to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(scan[:, :3])  # Use x, y, z coordinates

# Visualize the LiDAR point cloud
o3d.visualization.draw_geometries([pcd])

# import open3d as o3d
# import numpy as np
# import time

# # Load all LiDAR frames from KITTI sequence 00
# num_frames = len(dataset.timestamps)  # Total frames
# vis = o3d.visualization.Visualizer()
# vis.create_window()

# pcd = o3d.geometry.PointCloud()

# for i in range(num_frames):  # Loop through all frames
#     scan = dataset.get_velo(i)  # Load LiDAR frame
#     pcd.points = o3d.utility.Vector3dVector(scan[:, :3])  # Convert to Open3D format

#     if i == 0:
#         vis.add_geometry(pcd)
#     else:
#         vis.update_geometry(pcd)

#     vis.poll_events()
#     vis.update_renderer()
#     time.sleep(0.05)  # Adjust speed of animation

# vis.destroy_window()



### Implement Basic ICP SLAM

# ICP-based SLAM Explanation

## What This Code Does

- Loads **two LiDAR scans** (`pcd1` and `pcd2`).
- Uses **Iterative Closest Point (ICP)** to align `pcd2` to `pcd1`.
- Computes a **transformation matrix** that represents how `pcd2` should be moved to match `pcd1`.

## Transformation Matrix

The transformation matrix output looks like this:

\[
\begin{bmatrix} 
R_{11} & R_{12} & R_{13} & t_x \\ 
R_{21} & R_{22} & R_{23} & t_y \\ 
R_{31} & R_{32} & R_{33} & t_z \\ 
0 & 0 & 0 & 1 
\end{bmatrix}
\]

Where:
- **R** represents **rotation** (how the scan is rotated).
- **t** represents **translation** (how the scan is shifted).

## Why This is Useful

This matrix updates the robot’s **position** and **map**, helping it track movement over time. 🚀


In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  




def icp_slam(source, target, threshold=0.5):
    """Applies Iterative Closest Point (ICP) for SLAM with preprocessing"""
    
    
    source = source.voxel_down_sample(voxel_size=0.3)
    target = target.voxel_down_sample(voxel_size=0.3)

    
    target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=1.0, max_nn=30))

    
    icp = o3d.pipelines.registration.registration_icp(
        source, target, threshold, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=50)
    )
    
    
    translation = np.linalg.norm(icp.transformation[:3, 3])
    if translation > 1.5:  
        print("Warning: Large transformation detected, skipping frame")
        return np.eye(4)
    
    return icp.transformation



trajectory = [np.eye(4)]  


pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(dataset.get_velo(0)[:, :3])

num_frames = 4541  


for i in tqdm(range(1, num_frames), desc="Processing Frames"):
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(dataset.get_velo(i)[:, :3])

    transformation = icp_slam(pcd1, pcd2)

    trajectory.append(trajectory[-1] @ transformation)
    
    pcd1 = pcd2  


trajectory = np.array(trajectory)[:, :3, 3]  


plt.figure(figsize=(8, 6))
plt.plot(trajectory[:, 0], trajectory[:, 1], 'ro-', markersize=2, label="SLAM Path")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("SLAM Path using ICP")
plt.legend()
plt.grid()
plt.show()


### Ground Truth Path

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


gt_poses = np.loadtxt("/home/deepak-bhagat/Downloads/data_odometry_velodyne/dataset/poses/00.txt")
gt_poses = gt_poses.reshape(-1, 3, 4)


gt_x = gt_poses[:, 0, 3]  
gt_y = gt_poses[:, 2, 3]  
gt_trajectory = np.vstack((gt_x, gt_y)).T  


plt.figure(figsize=(8, 6))
plt.plot(gt_trajectory[:, 0], gt_trajectory[:, 1], 'g-', label='Ground Truth Path')

plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.legend()
plt.title("Ground Truth Trajectory")
plt.show()


### Slam vs Ground Path

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def rotate_trajectory(points, angle):
    """
    Rotates the given trajectory by a specified angle.

    Parameters:
        points (numpy.ndarray): (N, 2) array of (x, y) coordinates.
        angle (float): Rotation angle in radians.

    Returns:
        numpy.ndarray: Rotated trajectory.
    """
    R = np.array([[np.cos(angle), -np.sin(angle)], 
                  [np.sin(angle), np.cos(angle)]])
    return points @ R.T  


gt_poses = np.loadtxt("/home/deepak-bhagat/Downloads/data_odometry_velodyne/dataset/poses/00.txt")
gt_poses = gt_poses.reshape(-1, 3, 4)


gt_x = gt_poses[:, 0, 3]  
gt_y = gt_poses[:, 2, 3]  
gt_trajectory = np.vstack((gt_x, gt_y)).T  


trajectory = np.array(trajectory)  
trajectory = trajectory[:, :2]  


angle = np.arctan2(trajectory[-1, 1] - trajectory[0, 1], trajectory[-1, 0] - trajectory[0, 0]) - \
        np.arctan2(gt_trajectory[-1, 1] - gt_trajectory[0, 1], gt_trajectory[-1, 0] - gt_trajectory[0, 0])


rotated_gt_trajectory = rotate_trajectory(gt_trajectory, angle)


translation = trajectory[0] - rotated_gt_trajectory[0]  
rotated_gt_trajectory += translation  


plt.figure(figsize=(8, 6))
plt.plot(rotated_gt_trajectory[:, 0], rotated_gt_trajectory[:, 1], 'g-', label='Aligned Ground Truth')
plt.plot(trajectory[:, 0], trajectory[:, 1], 'r-', marker='o', linestyle='-', label='SLAM Path')

plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.legend()
plt.title("SLAM vs Ground Truth Trajectory (Aligned)")
plt.show()


### Implement A Path Planning*

In [ ]:
import heapq

def astar(grid, start, goal):
    """A* Algorithm for Path Planning"""
    rows, cols = len(grid), len(grid[0])
    heap = [(0, start)]
    came_from = {start: None}
    cost_so_far = {start: 0}

    while heap:
        _, current = heapq.heappop(heap)
        if current == goal:
            break
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            next_node = (current[0] + dx, current[1] + dy)
            if 0 <= next_node[0] < rows and 0 <= next_node[1] < cols:
                new_cost = cost_so_far[current] + 1
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost
                    heapq.heappush(heap, (priority, next_node))
                    came_from[next_node] = current
    return came_from

# Example grid-based path planning
grid = [[0]*10 for _ in range(10)]  # 10x10 empty grid
start, goal = (0, 0), (9, 9)
path = astar(grid, start, goal)
print("Path found:", path)

### Visualize and Evaluate Results


In [ ]:
import matplotlib.pyplot as plt

trajectory = np.array([[0, 0], [1, 2], [2, 3]])  # Example trajectory
plt.plot(trajectory[:, 0], trajectory[:, 1], marker="o", label="SLAM Path")
plt.legend()
plt.show()